In [7]:
from art.attacks.extraction import CopycatCNN
from art.estimators.classification import KerasClassifier
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model, Sequential
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [8]:
# Carga del modelo creado
use_model = load_model('./model')
load_x = np.load('x_val.npy')
load_y = np.loadtxt('y_val.txt', delimiter=',')
X_train, X_test, y_train, y_test_ = train_test_split(load_x, load_y, test_size=0.30, random_state=1)

2023-05-23 21:02:46.541286: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_22' id:697 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_22}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](bias_7, Identity_22)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [9]:
# Creacion del clasificador
keras_classifier = KerasClassifier(use_model)
copyCat = CopycatCNN(batch_size_fit=16, batch_size_query=16, nb_epochs=5,nb_stolen=500,classifier=keras_classifier)

In [10]:
sequential_model = Sequential()
sequential_model.add(Convolution2D(32, (3, 3), strides=(1, 1), input_shape=(256, 256, 3), activation='relu'))
sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
sequential_model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
sequential_model.add(Flatten())
sequential_model.add(Dense(512, activation='softplus'))
sequential_model.add(Dense(26, activation='sigmoid'))
sequential_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [11]:
model_stolen = KerasClassifier(model=sequential_model)

In [12]:
stolen_classifier = copyCat.extract(
  x=X_train, 
  y=y_train, 
  thieved_classifier=model_stolen
)

/Users/oscarsaravia/Documents/Universidad/security-data-science/SDS-LAB07/venv/lib/python3.11/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-05-23 21:02:51.019402: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1_2/Sigmoid' id:724 op device:{requested: '', assigned: ''} def:{{{node dense_1_2/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1_2/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-23 21:02:51.077941: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_3_1/kernel/Assign' id:977 op device:{requested: '', assigned: ''} def:{{{node dense_3_1/kernel/Assign}} = AssignVaria

Train on 500 samples


2023-05-23 21:02:55.664873: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_3/mul' id:1064 op device:{requested: '', assigned: ''} def:{{{node loss_3/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_3/mul/x, loss_3/dense_3_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-23 21:02:55.701083: W tensorflow/c/c_api.cc:304] Operation '{name:'training/Adam/dense_3_1/bias/m/Assign' id:1287 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense_3_1/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense_3_1/bias/m, training/Adam/dense_3_1/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the fu

Epoch 1/5
500/500 [==============================] - 18s 36ms/sample - loss: 947.7023 - accuracy: 0.8540
Epoch 2/5
500/500 [==============================] - 18s 36ms/sample - loss: 1.4851 - accuracy: 0.9480
Epoch 3/5
500/500 [==============================] - 20s 40ms/sample - loss: 0.8271 - accuracy: 0.9820
Epoch 4/5
500/500 [==============================] - 19s 37ms/sample - loss: 0.0033 - accuracy: 1.0000
Epoch 5/5
500/500 [==============================] - 18s 35ms/sample - loss: 1.8514e-04 - accuracy: 1.0000


In [13]:
score1 = keras_classifier._model.evaluate(x=X_test, y=y_test_)
score2 = stolen_classifier._model.evaluate(x=X_test,y=y_test_)
print(score1)
print(score2)

/Users/oscarsaravia/Documents/Universidad/security-data-science/SDS-LAB07/venv/lib/python3.11/site-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-05-23 21:04:28.189985: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_2/mul' id:800 op device:{requested: '', assigned: ''} def:{{{node loss_2/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul/x, loss_2/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-23 21:04:31.635854: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_3/mul' id:1064 op device:{requested: '', assigned: ''} def:{{{node loss_3/mul}} = Mul[T=DT_FLOAT, _has_manual_con

[32.65007080078125, 0.6616667]
[21.0602086130778, 0.75333333]


Discusión
<br>
* Desde nuestro punto de vista, el ataque se puede mejorar. Los resultados obtenidos se deben a la cantidad de epochs y el tamaño del modelo que se utilizó.